Importing the required libraries

In [ ]:
import numpy as np
import nltk
import string
import random

Import and reading the corpus

In [ ]:
f=open('chatbot.txt','r',errors = 'ignore')
raw_doc=f.read()
raw_doc=raw_doc.lower()#converts text to lowercase
nltk.download('punkt')#using the punkt tokenizer
nltk.download('wordnet')#using the wordnet dictionary
sent_tokens = nltk.sent_tokenize(raw_doc)#converts doc to list of sentences
word_tokens = nltk.word_tokenize(raw_doc)#converts doc to list of wc

Example of sentence tokens

In [ ]:
sent_tokens[:2]

Examples of word tokens

In [ ]:
word_tokens[:2]

Text processing

In [ ]:
lemmer = nltk.stem.WordNetLemmatizer()
#wordnet is a semantically-oriented dictionary of english included in NLTK
def LemTokens(tokens):
  return [lemmer.lemmatize(token) for token in tokens]
remove_punct_dict = dict((ord(punct), None) for punct in string.punctuation)
def LemNormalize(text):
  return LemTokens(nltk.word_tokenize(text.lower().translate(remove_punct_dict)))
  #sentence are not broken down into words & punctuation->WordNet

Defining the greeting function

In [ ]:
GREET_INPUTS = ("hello","hi","greetings","sup","what's up","hey")
GREET_RESPONSES = ["hi","hey","nods","hi there","hello","I am glad! You are talking to me"]
def greet(sentence):
  for word in sentence.split():
    if word.lower() in GREET_INPUTS:
      return random.choice(GREET_RESPONSES)#it will give a random o/p

Response generation

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer#tfidf term frequency->all words repeated,frequency of occurence words
#& universe document frequency->attach the occurence of rare words
from sklearn.metrics.pairwise import cosine_similarity#numbers to words

In [ ]:
def response(user_response):
  robo1_response=''
  TfidfVec = TfidfVectorizer(tokenizer=LemNormalize, stop_words='english')
  tfidf = TfidfVec.fit_transform(sent_tokens)
  vals = cosine_similarity(tfidf[-1], tfidf)
  idx=vals.argsort()[0][-2]
  flat = vals.flatten()
  flat.sort()
  req_tfidf = flat[-2]
  if(req_tfidf==0):
    robo1_response=robo1_response+"I am sorry! I don't understand you"
    return robo1_response
  else:
    robo1_response = robo1_response+sent_tokens[idx]
    return robo1_response

Defining Conversations start/end protocols

In [ ]:
flag=True
print("BOT: My name is Sri. Let's have a conversation!Also, if you want to exist any time,just type Bye!")
while(flag==True):
  user_response = input()
  user_response=user_response.lower()
  if(user_response!='bye'):
    if(user_response=='thanks' or user_response=='thank you'):
      flag=False
      print("BOT: You are welcome..")
    else:
      if(greet(user_response)!=None):
        print("BOT: "+greet(user_response))
      else:
        sent_tokens.append(user_response)
        word_tokens=word_tokens+nltk.word_tokenize(user_response)
        final_words=list(set(word_tokens))
        print("BOT: ",end="")
        print(response(user_response))
        sent_tokens.remove(user_response)
  else:
    flag=False
    print("BOT: GoodBye! Take Care <3 ")